# Statistical Pocketbook

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

First, read the spreadsheets into pandas dataframes:

In [ ]:
# General data

# Specify the path to the Excel file
file_path = '../data/StatisticalPocketbook/pb2023-section1.xlsx'

# Specify the sheet names
sheet_names = ['general', 'growth', 'empl_rate', 'unempl_rate', 'share_gross_value_added', 'share_empl', 'population', 'trade_import', 'trade_export', 'EU-world']

# Read the specified sheets into a dictionary of DataFrames
general_dfs = pd.read_excel(file_path, sheet_name=sheet_names, header=0, index_col=0)

# Results
for name, df in general_dfs.items():
    print(f"DataFrame for sheet {name}:\n", df.head())

Format the relevant data from "General" into 1 dataframe.

In [ ]:
# Add population density
general_dfs['general']['PopDensity'] = general_dfs['general']['Population'] / general_dfs['general']['Area']

# Keep only last year entries (most current) for "growth"(industrial production growth); employment and unemployment rate
general_dfs['growth'] = general_dfs['growth'].iloc[:, [-1]]
general_dfs['growth'].columns = ['indust_growth']
general_dfs['empl_rate'] = general_dfs['empl_rate'].iloc[:, [-1]]
general_dfs['empl_rate'].columns = ['empl_rate']

general_dfs['share_gross_value_added'].columns = ['GVA_agriculture', 'GVA_industry', 'GVA_services']
general_dfs['share_empl'].columns = ['emplshare_agriculture', 'emplshare_industry', 'emplshare_services']

general_dfs['trade_import'] = general_dfs['trade_import'].iloc[:, [0]]
general_dfs['trade_import'].columns = ['trade_import_w']
general_dfs['trade_export'] = general_dfs['trade_export'].iloc[:, [0]]
general_dfs['trade_export'].columns = ['trade_export_w']

# Concatenate the DataFrames by row index
chosen_dfs = [general_dfs['general'], general_dfs['growth'], general_dfs['empl_rate'], 
                        general_dfs['share_gross_value_added'], general_dfs['share_empl'], 
                        general_dfs['trade_import'], general_dfs['trade_export']]

# Compile everything into 1 dataframe
general_all = pd.concat(chosen_dfs, axis=1, ignore_index=False)
general_all = general_all.drop("EU-27")

Visualize the distributions:

In [ ]:
# Plot histograms for each column
general_all.hist(bins=15, figsize=(12, 8))  
plt.tight_layout()  # Ensure proper spacing between subplots
plt.show()

# Almost all distributions are skewed, investigating ther log-transformations
general_all_log = general_all.apply(lambda x: np.log(x) if np.issubdtype(x.dtype, np.number) else x)
general_all_log.hist(bins=15, figsize=(12, 8))  
plt.tight_layout()  # Ensure proper spacing between subplots
plt.show()

It is definitely beneficial to use log-transformations of all variables (except maybe empl_rate and emplshare_services) when including them in future models, that rely on normally distributed data.

Read the general "Transport" section.

In [ ]:
# Transport

# Specify the path to the Excel file
file_path = '../data/StatisticalPocketbook/pb2023_section21.xlsx'

# Specify the sheet names
sheet_names = ['limits', 'empl', 'entrpr', 'house_exp_type', 'price_index', 'tax_fuel', 'tax_otrans']

# Read the specified sheets into a dictionary of DataFrames
transport_dfs = pd.read_excel(file_path, sheet_name=sheet_names, header=0, index_col=0)

# Results
for name, df in transport_dfs.items():
    print(f"DataFrame for sheet {name}:\n", df.head())

Format the variables and merge

In [ ]:
# LIMITS - some entries have multiple values devided by /. We can take the mean from them.
# Define a function for the transformation of entries multiple limits to their mean
def mean_column(column):

    # Split values by '/' and convert to integers
    temp = column.apply(lambda x: [float(val) for val in str(x).split('/')])
    # Calculate mean for rows with multiple values
    temp = temp.apply(lambda x: np.mean(x) if len(x) > 1 else x[0])
    # Convert the column to a numeric data type
    temp = pd.to_numeric(temp, errors='coerce')
    return temp

# Apply the transformation to the specified columns
columns_to_transform = ['speed_l_cars_builtup_areas',
                        'speed_l_cars_outside_builtup_areas',
                        'speed_l_cars_motorways']

for column_name in columns_to_transform:
    transport_dfs['limits'][column_name] = mean_column(transport_dfs['limits'][column_name])

# Number of public transport enterprizes
transport_dfs['entrpr'] = transport_dfs['entrpr'].iloc[:, [2]]
transport_dfs['entrpr'].columns = ['pass_transport_enterpr']

# Transport expenses per head
transport_dfs['house_exp_type'] = transport_dfs['house_exp_type'].iloc[:, [-1]]
transport_dfs['house_exp_type'].columns = ['expenditure_per_head']

# Fuel taxes (as a % of GDP)
transport_dfs['tax_fuel'] = transport_dfs['tax_fuel'].iloc[:, [-3]]
transport_dfs['tax_fuel'].columns = ['tax_fuel']

# other Transport taxes (excl. fuel taxes)
transport_dfs['tax_otrans'] = transport_dfs['tax_otrans'].iloc[:, [-3]]
transport_dfs['tax_otrans'].columns = ['tax_otrans']

# Concatenate the DataFrames by row index
chosen_dfs = [transport_dfs['limits'], transport_dfs['entrpr'], transport_dfs['house_exp_type'], 
                        transport_dfs['tax_fuel'], transport_dfs['tax_otrans']]

# Compile everything into 1 dataframe
transport_all = pd.concat(chosen_dfs, axis=1, ignore_index=False)
transport_all = transport_all.drop("EU-27")

In [ ]:
# Plot histograms for each column
transport_all.hist(bins=15, figsize=(12, 8))  
plt.tight_layout()  # Ensure proper spacing between subplots
plt.show()

A lot of variables here have a distribution where 1 value is dominant due to EU-wide standards (like speed limits and blood alcohol levels). Tests/models that are not dependent on the normal distribution maybe should be considered.

In [ ]:
# Performance of Passenger Transport

# Specify the path to the Excel file
file_path = '../data/StatisticalPocketbook/pb2023_section23.xlsx'

# Specify the sheet names
sheet_names = ['perf_mode_pkm', 'perf_mode_split', 'split_mode_proz', 'split_mode_pkm', 'cars', 'bus_coach', 'tram_and_metro', 'rail_pkm', 'hs_rail']

# Read the specified sheets into a dictionary of DataFrames
passenger_dfs = pd.read_excel(file_path, sheet_name=sheet_names, header=0, index_col=0)

# Remove rows containing only NaN-s
for name, df in passenger_dfs.items():
    df.dropna(how='all', inplace=True)
    print(f"DataFrame for sheet {name}:\n", df.head())

From the "Performance of Passenger Transport" we have mainly the modal split by different factors expressed in pkm or in %.

First, out of this data we create a data frame modal_split, which includes data in million passenger-kilometers, that will be our main variable.

In [ ]:
modal_split = passenger_dfs['split_mode_pkm']
# Drop "Total" column for now
modal_split.drop(modal_split.columns[-1], axis=1, inplace=True)
# Remove countries without data
modal_split = modal_split.dropna(how='all')
# Replace all other NaN-s with 0
modal_split = modal_split.fillna(0)
# Rename columns
modal_split.columns = ['cars', 'bus_coach', 'railways', 'tram_metro']

Creating modal_split_proz (a data frame for modal split in percentage) separately.

In [ ]:
modal_split_proz = passenger_dfs['split_mode_proz']
modal_split_proz.set_index('Unnamed: 1', inplace=True)
# Remove countries without data
modal_split_proz = modal_split_proz.dropna(how='all')
# Replace all other NaN-s with 0
modal_split_proz = modal_split_proz.fillna(0)
# Rename columns
modal_split_proz.columns = ['cars', 'bus_coach', 'railways', 'tram_metro']

In [ ]:
# Plot the modal split as a bar chart
modal_split_proz.plot(kind='bar', stacked=True, figsize=(12, 8))

# Customize the plot
plt.xlabel('Country')
plt.ylabel('Percentage (%)')
plt.title('Modal Split of Passenger Transport by Country and Means of Transport (2021)')
plt.xticks(rotation=90)

# Show the plot
plt.tight_layout()
plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Sort the DataFrame by the 'tram_metro' column in descending order
modal_split_sorted = modal_split_proz.sort_values(by='tram_metro', ascending=False)

# Plot the modal split as a bar chart
modal_split_sorted.plot(kind='bar', stacked=True, figsize=(12, 8))

# Customize the plot
plt.xlabel('Country')
plt.ylabel('Percentage (%)')
plt.title('Modal Split of Passenger Transport by Country and Means of Transport (2021) - Ordered by Tram & Metro')
plt.xticks(rotation=90)

# Show the plot
plt.tight_layout()
plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
plt.show()


In [ ]:
# Infrastructure

# Specify the path to the Excel file
file_path = '../data/StatisticalPocketbook/pb2023_section25.xlsx'

# Specify the sheet names
sheet_names = ['motorway', 'length_road', 'rail_length', 'airports', 'ports', 'length_oil']

# Read the specified sheets into a dictionary of DataFrames
infrastructure_dfs = pd.read_excel(file_path, sheet_name=sheet_names, header=4, index_col=1)

# Results
for name, df in infrastructure_dfs.items():
    print(f"DataFrame for sheet {name}:\n", df.head())

In [ ]:
infrastructure_dfs['motorway'] = infrastructure_dfs['motorway'].iloc[:, [-1]]
infrastructure_dfs['motorway'].columns = ['len_motorway']

infrastructure_dfs['length_road']['len_total_road'] = infrastructure_dfs['length_road'].sum(axis=1, skipna=True)
infrastructure_dfs['length_road'] = infrastructure_dfs['length_road'].iloc[:, [-1]]

infrastructure_dfs['rail_length'] = infrastructure_dfs['rail_length'].iloc[:, [-2]]
infrastructure_dfs['rail_length'].columns = ['len_rail']

infrastructure_dfs['length_oil'] = infrastructure_dfs['length_oil'].iloc[:, [-1]]
infrastructure_dfs['length_oil'].columns = ['len_oil']

# Coose dataframes to compile
chosen_dfs = [infrastructure_dfs['motorway'], infrastructure_dfs['length_road'], infrastructure_dfs['rail_length'], 
                        infrastructure_dfs['length_oil']]

# Compile everything into 1 dataframe
infrastructure_all = pd.concat(chosen_dfs, axis=1, ignore_index=False)
infrastructure_all = infrastructure_all.drop("EU27")

In [ ]:
# Plot histograms for each column
infrastructure_all.hist(bins=15, figsize=(8, 5))  
plt.tight_layout()  # Ensure proper spacing between subplots
plt.show()

# Investigating log-transformations
infrastructure_all_log = infrastructure_all.apply(lambda x: np.log(x) if np.issubdtype(x.dtype, np.number) else x)
infrastructure_all_log.hist(bins=15, figsize=(8, 5))  
plt.tight_layout()  # Ensure proper spacing between subplots
plt.show()

In [ ]:
# Means of Transport

# Specify the path to the Excel file
file_path = '../data/StatisticalPocketbook/pb2023_section26.xlsx'

# Specify the sheet names
sheet_names = ['stock_cars', 'stock_busses']

# Read the specified sheets into a dictionary of DataFrames
means_transp_dfs = pd.read_excel(file_path, sheet_name=sheet_names, header=4, index_col=1)

# Remove rows containing only NaN-s
for name, df in means_transp_dfs.items():
    df.dropna(how='all', inplace=True)
    print(f"DataFrame for sheet {name}:\n", df.head())

In [ ]:
# Number of registered cars
means_transp_dfs['stock_cars'] = means_transp_dfs['stock_cars'].iloc[:, [-1]]
means_transp_dfs['stock_cars'].columns = ['stock_cars']

# Number of registered busses
means_transp_dfs['stock_busses'] = means_transp_dfs['stock_busses'].iloc[:, [-1]]
means_transp_dfs['stock_busses'].columns = ['stock_busses']

# Compile to 1 dataframe:
chosen_dfs = [means_transp_dfs['stock_cars'], means_transp_dfs['stock_busses']]
means_transp_all = pd.concat(chosen_dfs, axis=1, ignore_index=False)
means_transp_all = means_transp_all.drop("EU-27")


In [ ]:
# Plot histograms for each column
means_transp_all.hist(bins=15, figsize=(8, 3))  
plt.tight_layout()  
plt.show()

# Look at log-distribution
np.log(means_transp_all).hist(bins=15, figsize=(8, 3))  
plt.tight_layout()  
plt.show()

In [ ]:
# Save all dfs into files
general_all.to_csv('../data/StatisticalPocketbook/general_all.csv')
transport_all.to_csv('../data/StatisticalPocketbook/transport_all.csv')
infrastructure_all.to_csv('../data/StatisticalPocketbook/generainfrastructure_alll_all.csv')
means_transp_all.to_csv('../data/StatisticalPocketbook/means_transp_all.csv')

# Modal split
modal_split.to_csv('../data/StatisticalPocketbook/modal_split.csv')
modal_split_proz.to_csv('../data/StatisticalPocketbook/modal_split_proz.csv')